# **Baseline CSV Dataset**

## Importação de pacotes e leitura dos dados

In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


### Função auxiliar para separação dos conjuntos de dados

In [29]:
def get_features_target_data(df, indexes_path):

    indexes = pd.read_csv(indexes_path)

    filtered_df = df.filter(items = indexes.sample_index, axis=0)

    X = filtered_df.drop(columns= 'label')
    y = filtered_df['label']

    return X, y

def get_train_val_set_data(df):

    X_train, y_train = get_features_target_data(df, '../dataset/Data/train_indexes.csv')
    X_val, y_val = get_features_target_data(df, '../dataset/Data/val_indexes.csv')
    X_test, y_test = get_features_target_data(df, '../dataset/Data/test_indexes.csv')

    return X_train, y_train, X_val, y_val, X_test, y_test

### Leitura dos dados e separação em treino, validação e test

In [31]:
dataset = pd.read_csv('../dataset/Data/features_30_sec.csv')

In [30]:
X_train, y_train, X_val, y_val, X_test, y_test = get_train_val_set_data(dataset)

## Implementação do baseline

In [36]:
features = dataset.columns.drop(['filename','label'])
X_train = X_train.drop(columns= 'filename')
X_val = X_val.drop(columns= 'filename')
X_test = X_test.drop(columns= 'filename')

---
**PERGUNTA - É um problema desbalanceado?**: 

Para o problema proposto, nota-se que as classes no treino estão bem distribuídas entre si. Assim, não identifica-se um problema desbalanceado

In [35]:
y_train.value_counts(normalize = True)

blues        0.109046
hiphop       0.106568
country      0.102850
jazz         0.100372
pop          0.099133
classical    0.097893
disco        0.097893
rock         0.097893
reggae       0.094176
metal        0.094176
Name: label, dtype: float64

---
**PERGUNTA - Os dados precisarão de algum tipo de pré-processamento?**: 

No caso do arquivo csv será necessário realizar apenas a normalização pois nosso baseline trata-se de uma SVM. Para os arquivos de imagem, é necessário fazer a extração de cores por pixel. Por fim, no caso dos arquivos de áudio, é necessário convertê-los em alguma forma interpretável como uma sequência de inteiros por frame da música.

### 5. **Implementação do Baseline**

Como Baseline, optou-se pelo uso de uma SVM para classificação. No caso, não foram implementadas otimizações de hiperparâmetros. A escolha da SVM foi feita pois o número de amostras disponíveis não é alto e dito algoritmo é robusto a um alto número de features

In [37]:
from sklearn.svm import SVC
from sklearn import metrics

In [38]:
clfBaseline = SVC()

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

scaledXtrain = scaler.transform(X_train)
scaledXval = scaler.transform(X_val)

In [40]:
clfBaseline.fit(scaledXtrain, y_train)

SVC()

---
**PERGUNTA - Quais as métricas utilizadas neste problema? Acurácia, erro, MSRE, etc.? O que elas significam?**: 

Para o problema proposto é possível a aplicação de métricas como acurácia balanceada, recall, precisão e f1-score. O significado de cada uma é apresentado abaixo:

1.   Acurácia: Média de TP/(TP + TN + FP + FN) entre as classes
2.   Recall: TP / FN + TP
3.   Precisão: TP / TP + FP
4.   F1-Score: Média harmônica de recall e precisão



In [41]:
resultados = clfBaseline.predict(scaledXval)
print(metrics.classification_report(y_val,resultados))

              precision    recall  f1-score   support

       blues       0.67      0.89      0.76         9
   classical       0.88      0.88      0.88        16
     country       0.67      0.83      0.74        12
       disco       0.89      0.53      0.67        15
      hiphop       0.83      0.45      0.59        11
        jazz       0.79      0.79      0.79        14
       metal       0.88      0.88      0.88        17
         pop       0.80      0.71      0.75        17
      reggae       0.61      0.74      0.67        19
        rock       0.38      0.46      0.41        13

    accuracy                           0.72       143
   macro avg       0.74      0.72      0.71       143
weighted avg       0.75      0.72      0.72       143



In [42]:
print(metrics.balanced_accuracy_score(y_val,resultados))

0.7157431156734562


Avaliando os resultados acima, nota-se um resultado muito satisfatório para o Baseline. As métricas de acurácia, recall e precisão ficaram em torno de ~0.7 o que é muito positivo dado o grau de complexidade limitado para o modelo.